In [1]:
import os

In [2]:
%pwd

'c:\\Users\\KUNAL MEHTA\\Desktop\\Data Science Training\\Projects\\Auto-Insurance-Risk-Profiling\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\KUNAL MEHTA\\Desktop\\Data Science Training\\Projects\\Auto-Insurance-Risk-Profiling'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PredictionsConfig:
    root_dir: Path
    test_data_path: Path
    threshold_path: Path
    class_model_path: Path
    reg_model_path: Path
    potential_customers_data_path: Path
    potential_customers_with_predictions_data_path: Path


In [6]:
from AutoInsurance.constants import *
from AutoInsurance.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_predictions_config(self) -> PredictionsConfig:
        config = self.config.predictions
        predictions_config =  PredictionsConfig(
            root_dir = Path(config.root_dir),
            threshold_path = Path(config.threshold_path),
            test_data_path = Path(config.test_data_path),
            class_model_path = Path(config.class_model_path),
            reg_model_path = Path(config.reg_model_path),
            potential_customers_data_path = Path(config.potential_customers_data_path)
            potential_customers_with_predictions_data_path = Path(config.potential_customers_with_predictions_data)
        )
    
        return predictions_config

In [23]:
import joblib 
import pandas as pd
from AutoInsurance.utils.common import load_json
import numpy as np
from AutoInsurance.utils.common import logger

In [27]:
class Predictions:
    def __init__(self, config: PredictionsConfig):
        self.config = config

    def get_predictions(self):

        logger.info(f"class_model_path: {self.config.class_model_path}")
        logger.info(f"reg_model_path: {self.config.reg_model_path}")
        logger.info(f"test_data_path: {self.config.test_data_path}")
        logger.info(f"threshold_path: {self.config.threshold_path}")
        logger.info(f"potential_customers_data_path: {self.config.potential_customers_data_path}")


        class_model = joblib.load(Path(self.config.class_model_path))
        reg_model = joblib.load(Path(self.config.reg_model_path))

        test_data = pd.read_csv(Path(self.config.test_data_path))
        class_predictions_probs = class_model.predict_proba(test_data)[:, 1]

        # print(Path(self.config.threshold_path))

        metrics = load_json(Path(self.config.threshold_path))
        class_predictions = (class_predictions_probs >= metrics.optimal_threshold).astype(int)

        reg_predictions = reg_model.predict(test_data)
        reg_predictions = np.expm1(reg_predictions)

        potential_customers_data = pd.read_csv(Path(self.config.potential_customers_data_path))
        potential_customers_data['claim_probability'] = class_predictions_probs
        potential_customers_data['claim'] = class_predictions
        potential_customers_data['claim_amount'] = reg_predictions

        
        # print(potential_customers_data.head())
        

        # potential_customers_data.to_csv(Path(self.config.potential_customers_data_path), index = False)


In [28]:
config = ConfigurationManager()
predictions_config = config.get_predictions_config()
predictions_config = Predictions(config = predictions_config)
predictions_config.get_predictions()

[2024-05-23 16:28:38,191: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-23 16:28:38,194: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-23 16:28:38,198: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-23 16:28:38,200: INFO: common: created directory at: artifacts]
[2024-05-23 16:28:38,201: INFO: 4178703687: class_model_path: artifacts\model_trainer\class_model.joblib]
[2024-05-23 16:28:38,201: INFO: 4178703687: reg_model_path: artifacts\model_trainer\reg_model.joblib]
[2024-05-23 16:28:38,202: INFO: 4178703687: test_data_path: artifacts\data_transformation\Processed_test_data.csv]
[2024-05-23 16:28:38,202: INFO: 4178703687: threshold_path: artifacts\model_evaluation\class_metrics.json]
[2024-05-23 16:28:38,203: INFO: 4178703687: potential_customers_data_path: artifacts\data_transformation\potential_customers.csv]
[2024-05-23 16:28:38,293: INFO: common: json file loaded succesfully from: artifacts\model_evaluation\class_